<a href="https://colab.research.google.com/github/jsebastianwrc/ProyectoFinal_IA_IS2/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas
filename = '1987 - 2022.csv'
data = pandas.read_csv(filename, header=0)

print(data.head(100))

                                 Title Seasons Episodes  \
0                        ¡Mucha Lucha!       3       52   
1                                 10+2       1       52   
2   100% Wolf: Legend of the Moonstone       2       26   
3                          1001 Nights       2       52   
4                 101 Dalmatian Street       1       40   
..                                 ...     ...      ...   
95                   Angry Birds Toons       3      104   
96         Angry Birds: Summer Madness       3       36   
97                           Angry Kid       4       66   
98                     Animal Crackers       3       52   
99                  Animal Mechanicals       3       37   

                                           Country  Premiere Year Final Year  \
0                                    United States           2002       2005   
1                                            Spain           1994       1994   
2                                        Australia 